In [6]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

In [7]:
# ФОРМАТ DATETIME

# Таким форматом в Pandas является формат datetime,
# который записывается как YYYY-MM-DD HH: MM: SS,
# то есть составляющие времени указываются
# в следующем порядке: год, месяц, день, час, минута, секунда.

# В наших данных дата записана в виде DD/MM/YYYY,
# например 3/12/2017. Посмотрим на это:

display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

In [8]:
# Для того чтобы преобразовывать столбцы с датами,
# записанными в распространённых форматах, в формат datetime,
# можно воспользоваться
# функцией pandas.to_datetime().
# В нашем случае в функции нужно указать
# параметр dayfirst=True, который будет обозначать,
# что в первоначальном признаке первым идет день.
# Преобразуем столбец Date в формат datetime, передав его в эту функцию:

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

In [10]:
# ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

# Тип данных datetime позволяет с помощью специального
# аксессора dt 
# выделять составляющие времени из каждого элемента столбца, такие как:

# date — дата;
# year, month, day — год, месяц, день;
# time — время;
# hour, minute, second — час, минута, секунда;
# dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
# day_name — название дня недели;
# dayofyear — порядковый день года;
# quarter — квартал (интервал в три месяца).

# Аксессор — это атрибут столбца, хранящий переменные, 
# которые были строковым представлением времени,
# а затем были изменены с помощью pd.to_datetime().

# Обратите внимание, что вы не сможете обратиться к аксессору,
# если ваш столбец не приведён к типу datetime.

# Например, обратившись по атрибуту dt.year в столбце Date, 
# мы можем «достать» год продажи и понять, за какой интервал
# времени (в годах) представлены наши данные, а также на 
# какой год приходится наибольшее число продаж:

years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

# В результате обращения к атрибуту datetime melb_df['Date'].dt.year 
# мы получаем объект Series, 
# в котором в качестве значений выступают годы продажи объектов недвижимости.
# Мы можем занести результат в переменную year_sold 
# и далее работать с ней как с обычным столбцом Series
# — вычислять максимум, минимум и модальное значение.

#Из результатов видно,
# что данные представлены за интервал с 2016 по 2017 год
# и наибольшее количество объектов было продано в 2017 году.

# Примечание. 
# Так как модальных значений в столбце может быть несколько,
# метод mode() возвращает объект Series,
# даже если мода в данных только одна.
# Чтобы сохранить стилистику вывода информации о годе продажи
# и выводить только число, а не Series,
# мы обращаемся к результату работы метода mode() по индексу 0.

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int32
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


In [11]:
# Теперь попробуем понять, на какие месяцы приходится пик продаж
# объектов недвижимости. Для этого выделим атрибут dt.month
# и на этот раз занесём результат в столбец MonthSale,
# а затем найдём относительную частоту продаж для каждого месяца 
# от общего количества продаж — 
# для этого используем метод value_counts()
# с параметром normalize (вывод в долях):

# Как правило, результаты заносятся в таблицу (в новый столбец),
# когда предполагается, что столбец будет использоваться в дальнейшем.
# Промежуточные и тренировочные результаты будем записывать в переменные.

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

# Из результатов становится ясно, что наибольшее количество продаж 
# недвижимости приходится на май, июль и сентябрь 
# (пятый, седьмой и девятый месяцы соответственно). 
# Месяцами застоя при этом являются месяцы
# — октябрь, февраль и январь (десятый, второй и первый 
# месяцы соответственно).

MonthSale
5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: proportion, dtype: float64

In [12]:
# РАБОТА С ИНТЕРВАЛАМИ

# Часто бывает такая ситуация, что необходимо
# вычислять интервалы между двумя временными промежутками.
# Например, можно вычислить, сколько дней прошло 
# с 1 января 2016 года до момента продажи объекта. 
# Для этого можно просто найти разницу между датами 
# продаж и заявленной датой, представленной в формате datetime:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

In [13]:
# В результате мы получаем Series,
# элементами которой является количество дней,
# которое прошло с 1 января 2016 года.
# Обратите внимание, что данные такого формата
# относятся к типу
# timedelta.

# Чтобы превратить количество дней 
# из формата интервала в 
# формат целого числа дней, 
# можно воспользоваться аксессором dt
# для формата timedelta и извлечь из него
# атрибут days:

display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int32

In [12]:
# Рассмотрим другой пример. 
# Давайте создадим признак
# возраста объекта недвижимости в годах на момент продажи.
# Для этого выделим из столбца с датой продажи год 
# и вычтем из него год постройки здания.
# Результат оформим в виде столбца AgeBuilding:

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

In [14]:
# Примечание. Обратите внимание, что, так как года кодируются целым числом,
# в результате мы тоже получаем целочисленный столбец
# — тип int64 (а не timedelta).

# На самом деле столбец AgeBuilding дублирует
# информацию столбца YearBuilt,
# так как, зная год постройки здания, мы автоматически знаем его возраст.
# Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания,
# так как он является более наглядным, а год постройки удалим из таблицы:

melb_df = melb_df.drop('YearBuilt', axis=1)

In [15]:
# Задание 3.3
# 
# Создайте в таблице melb_df признак WeekdaySale (день недели).
# 
# Найдите, сколько объектов недвижимости было продано 
# в выходные (суббота и воскресенье),
# результат занесите в переменную weekend_count.
# В качестве ответа введите результат вывода переменной weekend_count.

melb_df['WeekdaySale'] = melb_df['Date'].dt.day_of_week
weekend_count = melb_df[melb_df['WeekdaySale'] > 4].shape[0]
display(weekend_count)

12822

In [16]:
%store melb_df

Stored 'melb_df' (DataFrame)


In [17]:
# ВВОДНЫЕ ДАННЫЕ ДЛЯ ВЫПОЛНЕНИЯ ЗАДАНИЙ 3.4-3.5

# Вам представлены данные (в формате csv) об отчётах
# очевидцев НЛО в США за период с 1930 по 2020 год.

#В данных есть следующие признаки:

# "City" — город, где был замечен НЛО;
# "Colors Reported" — цвет объекта;
# "Shape Reported" — форма объекта;
# "State" — обозначение штата;
# "Time" — время, когда был замечен НЛО 
#           (данные отсортированы от старых наблюдений к новым). 

# Прочитайте данные,
# сделайте преобразование времени к формату datetime
# и выполните задания ниже.

ufo_data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep=',')
ufo_data['Time'] = pd.to_datetime(ufo_data['Time'])
ufo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18241 entries, 0 to 18240
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   City             18215 non-null  object        
 1   Colors Reported  2882 non-null   object        
 2   Shape Reported   15597 non-null  object        
 3   State            18241 non-null  object        
 4   Time             18241 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(4)
memory usage: 712.7+ KB


In [27]:
# Задание 3.4

# В каком году отмечается наибольшее количество 
# случаев наблюдения НЛО в США?

ufo_data['Time'].dt.year.value_counts().nlargest(1).index



Index([1999], dtype='int32', name='Time')

In [32]:
ufo_data.head()

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00


In [35]:
# Задание 3.5

# Найдите средний интервал времени (в днях)
# между двумя последовательными случаями наблюдения НЛО
# в штате Невада (NV).

# Чтобы выделить дату из столбца Time, 
# можно воспользоваться атрибутом datetime date.

# Чтобы вычислить разницу между двумя соседними датами в столбце,
# примените к нему метод diff().

# Чтобы перевести интервал времени в дни, 
# воспользуйтесь атрибутом timedelta days.

x = ufo_data[ufo_data['State'] == 'NV']['Time'].dt.date.diff().dt.days.mean()
round(x)

69